# Myelin Mapping Notebook

These workflows process T1 and T2 weighted images to then be used to quantify cortical myelin content per methods described in Gordon et al., 2017 (doi: 10.1016/j.neuron.2017.07.011)

In [1]:
# Import all othe functions/nodes being used
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl import FSLCommand, BET, Reorient2Std, FAST
from nipype.interfaces.freesurfer import BBRegister, Binarize, ReconAll
from nipype.interfaces.ants import N4BiasFieldCorrection
from pandas import read_csv

# Declare study-specific variables
setup='Cat'

if setup=='Jay':
    study_home = '/Users/dendrite/Desktop/Jay'
    raw_data = study_home + '/raw'
elif setup=='Cat':
    study_home = '/moochie/user_data/CamachoCat/5YOP'
    raw_data = '/moochie/study_data/5YOP/MRI_processing'
    
output_dir = study_home + '/proc'
workflow_dir = study_home + '/workflows'
fs_dir = study_home + '/freesurfer'
session_info = read_csv(study_home + '/misc/session_info.csv',index_col=None)

subject_ids = session_info['subject_id'].unique().tolist()
subject_ids = map(str,subject_ids)

# Set any processing settings
FSLCommand.set_default_output_type('NIFTI_GZ')

200102-14:53:26,635 nipype.utils INFO:
	 Running nipype version 1.2.3 (latest: 1.4.0)


In [2]:
# Identity node- select subjects
infosource = Node(IdentityInterface(fields=['subject_id']),
                     name='infosource')
infosource.iterables = ('subject_id', subject_ids)

# Selectfiles node to grab subject T2w data
t2w_templates = {'T2anat': raw_data + '/{subject_id}/ANAT_NIFTIS/t2w_2.nii.gz'}
t2w_source = Node(SelectFiles(t2w_templates), name='t2w_source')

# datasink node
datasink = Node(DataSink(base_directory=output_dir), name='datasink')

## Preprocess T1w data

In [3]:
# grab raw T1-weighted images and process through freesurfer
t1w_template = {'t1_anats':raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz'}
t1w_source = Node(DataGrabber(base_directory = raw_data, 
                              field_template = t1w_template,
                              template=raw_data + '/%s/ANAT_NIFTIS/t1w_*.nii.gz',
                              sort_filelist=True,
                              in_fields=['subject_id'],
                              outfields=['t1_anats'],
                              template_args={'t1_anats':[['subject_id']]}), 
                  name='t1w_source')

#process through recon-all
reconall = Node(ReconAll(directive='all',flags='-gcut',openmp=4,
                         subjects_dir=fs_dir,use_T2=True), name='reconall')

In [ ]:
fs_proc = Workflow(name='freesurfer_proc_flow')
fs_proc.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                 (infosource, t2w_source, [('subject_id','subject_id')]),
                 (infosource, reconall, [('subject_id','subject_id')]),
                 (t1w_source, reconall, [('t1_anats','T1_files')]), 
                 (t2w_source, reconall, [('T2anat','T2_file')])
                ])
fs_proc.base_dir = workflow_dir
fs_proc.write_graph(graph2use='flat')
fs_proc.run('MultiProc', plugin_args={'n_procs': 10, 'memory_gb': 20})

200102-14:53:30,526 nipype.workflow INFO:
	 Generated workflow graph: /moochie/user_data/CamachoCat/5YOP/workflows/freesurfer_proc_flow/graph.png (graph2use=flat, simple_form=True).
200102-14:53:30,546 nipype.workflow INFO:
	 Workflow freesurfer_proc_flow settings: ['check', 'execution', 'logging', 'monitoring']
200102-14:53:30,577 nipype.workflow INFO:
	 Running in parallel.
200102-14:53:30,581 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 20.00/20.00, Free processors: 3/3.
200102-14:53:30,697 nipype.workflow INFO:
	 [Node] Setting-up "freesurfer_proc_flow.t2w_source" in "/moochie/user_data/CamachoCat/5YOP/workflows/freesurfer_proc_flow/_subject_id_4000/t2w_source".
200102-14:53:30,698 nipype.workflow INFO:
	 [Node] Setting-up "freesurfer_proc_flow.t2w_source" in "/moochie/user_data/CamachoCat/5YOP/workflows/freesurfer_proc_flow/_subject_id_3000/t2w_source".
200102-14:53:30,699 nipype.workflow INFO:
	 [Node] Setting-up "freesurfer_proc_flow.t

## Preprocess T2w data
The nodes and workflow below preprocess T2-weighted images to match T1-weighted data processed externally using FreeSurfer v6.

In [ ]:
# FreeSurferSource node to grab processed T1w data from freesurfer
t1w_source = Node(FreeSurferSource(subject_dir=fs_dir), name='t1w_source')

# skullstrip the T2w using BET (FSL)
t2strip = Node(BET(), name='t2strip')

# N4 bias correction of the T2w data (ANTs)
bias_correct = Node(N4BiasFieldCorrection(),name='bias_correct')

# Reorient T2w to standard
reorient_t2w = Node(Reorient2Std(), name='reorient_t2w')

# Register T2w to T1w using FreeSurfer's Boundary-Based Registration
register = Node(BBRegister(subjects_dir = fs_dir, contrast_type='t2'), name='register')

In [ ]:
# Write the workflow
preprocess_t2 = Workflow(name='preprocess_t2')
preprocess_t2.connect([(infosource, t1w_source, [('subject_id','subject_id')]),
                       (infosource, t2w_source, [('subject_id','subject_id')]),
                       (infosource, register, [('subject_id','subject_id')]),
                       (t2w_source, t2strip,[('T2_anat','in_file')]),
                       (t2strip, reorient_t2w,[('out_file','in_file')]),
                       (reorient_t2w, bias_correct,[('out_file','input_image')]),
                       (bias_correct, register,[('output_image','source_file')]),
                       (t2strip, datasink,[('out_file','skullstripped_t2w')]),
                       (register, datasink,[('registered_file','registered_t2w')])
                       ])

preprocess_t2.base_dir = workflow_dir
preprocess_t2.write_graph(graph2use='flat')
preprocess_t2.run('MultiProc', plugin_args={'n_procs': 2, 'memory_gb': 10})

## Resample surfaces

In [ ]:
def resample_surfs(surfs):
    return(resampled_surfs)

def volume_to_surface_myelin(volume):
    return(surface_map)


## Create within subject template and masks

In [ ]:
def make_non_gm_mask(tissue_class_map):
    from nipype import config, logging
    config.enable_debug_mode()
    logging.update_logging(config)
    from os.path import abspath
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    
    wm_val = 1
    csf_val = 2
    seg_img = load(tissue_class_map)
    seg_data = seg_img.get_data()
    new_seg_data = zeros_like(seg_data)
    new_seg_data[seg_data==wm_val] = 1
    new_seg_data[seg_data==csf_val] = 1
    
    new_seg_img = Nifti1Image(new_seg_data,header=seg_img.header,affine=seg_img.affine)
    save(new_seg_img,'non_gm_mask.nii.gz')
    nongm_mask = abspath('non_gm_mask.nii.gz')
    
    return(nongm_mask)
    

In [ ]:
grab_t1ws = Node(SelectFiles(t1w_template), name='grab_t1ws')

make_avg = Node(Function(input_names=['in_files','num_proc','dimensions'], 
                         output_names=['template_file'],
                         function=create_template),
                name='make_avg')
make_avg.inputs.num_proc=6
make_avg.inputs.dimensions=3

create_mask = Node(BET(mask='mask.nii.gz',no_output=True), name='create_mask')

dilate_mask = Node(Binarize(min=0.5, dilate=1, binary_file='mask_D1.nii.gz'),name='dilate_mask')

segment_brain = Node(FAST(number_classes=3), name='segment_brain')

make_nongm_mask = Node(Function(input_names=['tissue_class_map'],
                                output_names=['nongm_mask'],
                                function=make_non_gm_mask), 
                       name='make_nongm_mask')